In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define Dataset class to wrap the preprocessed dataset
class PreprocessedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset["label"])

    def __getitem__(self, idx):
        return {
            "label": self.dataset["label"][idx],
            "pixel_values": torch.tensor(self.dataset["pixel_values"][idx]),  # Ensure it's a tensor
        }


In [ ]:
# Load preprocessed datasets into PyTorch Dataloaders
train_dataset = PreprocessedDataset(preprocessed_train_ds)
val_dataset = PreprocessedDataset(preprocessed_val_ds)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

NameError: name 'preprocessed_train_ds' is not defined

In [ ]:
# Initialize CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

In [ ]:
class CLIPForImageClassification(nn.Module):
    def __init__(self, num_labels=10):
        super(CLIPForImageClassification, self).__init__()
        # Load the CLIP model
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.projection_dim = self.clip.config.projection_dim
        
        # Freeze CLIP parameters
        for param in self.clip.parameters():
            param.requires_grad = False
            
        # Simple classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.projection_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )
        self.num_labels = num_labels

    def forward(self, pixel_values, labels=None):
        with torch.no_grad():  # Don't compute gradients for CLIP
            outputs = self.clip.get_image_features(pixel_values=pixel_values)
        
        logits = self.classifier(outputs)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=None,
            attentions=None
        )


In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
from evaluate import load

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
import torch

num_epochs = 6
batch_size = 8
learning_rate = 2e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPForImageClassification()
from torch.utils.data import DataLoader

train_loader = DataLoader(preprocessed_train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(preprocessed_val_ds, batch_size=batch_size, shuffle=False, num_workers=4)
from transformers import TrainingArguments, Trainer

metric_name = "accuracy"

args = TrainingArguments(
    f"test-clothing-clip",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
)
from transformers import default_data_collator

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=preprocessed_train_ds,
    eval_dataset=preprocessed_val_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/966 [00:00<?, ?it/s]

  0%|          | 0/137 [00:00<?, ?it/s]

{'eval_loss': 1.8049569129943848, 'eval_accuracy': 0.35714285714285715, 'eval_runtime': 33.5939, 'eval_samples_per_second': 16.253, 'eval_steps_per_second': 4.078, 'epoch': 1.0}


ValueError: You are trying to save a non contiguous tensor: `clip.visual_projection.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [ ]:
class CLIPForImageClassification(nn.Module):
    def __init__(self, num_labels=10, dropout_rate=0.1):
        super(CLIPForImageClassification, self).__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.clip.config.projection_dim, num_labels)
        self.num_labels = num_labels
        
        # Freeze CLIP parameters
        for param in self.clip.parameters():
            param.requires_grad = False

    def forward(self, pixel_values, labels=None):
        # Get the image features from CLIP
        with torch.no_grad():
            outputs = self.clip.get_image_features(pixel_values=pixel_values)
        
        # Apply dropout to the features
        features = self.dropout(outputs)
        
        # Pass through the classification layer
        logits = self.classifier(features)

        # Compute loss only if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Return a SequenceClassifierOutput object
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=None,  # CLIP doesn't return hidden states in this usage
            attentions=None,     # CLIP doesn't return attention in this usage
        )

OSError: [Errno 22] Invalid argument

In [ ]:
class CLIPForImageClassification(nn.Module):
    def __init__(self, num_labels=10, dropout_rate=0.1):
        super(CLIPForImageClassification, self).__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.clip.config.projection_dim, num_labels)
        self.num_labels = num_labels
        
        # Freeze CLIP parameters
        for param in self.clip.parameters():
            param.requires_grad = False

    def forward(self, pixel_values, labels=None):
        # Get the image features from CLIP
        with torch.no_grad():
            outputs = self.clip.get_image_features(pixel_values=pixel_values)
        
        # Apply dropout to the features
        features = self.dropout(outputs)
        
        # Pass through the classification layer
        logits = self.classifier(features)

        # Compute loss only if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Return a SequenceClassifierOutput object
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=None,  # CLIP doesn't return hidden states in this usage
            attentions=None,     # CLIP doesn't return attention in this usage
        )

In [ ]:
preprocessed_test_ds

Dataset({
    features: ['label', 'img', 'pixel_values'],
    num_rows: 528
})

In [ ]:
from transformers import CLIPModel, CLIPProcessor
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

class CLIPForImageClassification(nn.Module):
    def __init__(self, num_labels=10):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.num_labels = num_labels
        self.classifier = nn.Linear(512, num_labels)
        
        # Freeze CLIP
        for param in self.clip.parameters():
            param.requires_grad = False

    def forward(self, pixel_values, labels=None):
        # Ensure pixel_values is the right shape
        if not isinstance(pixel_values, torch.Tensor):
            raise ValueError(f"pixel_values must be a tensor, got {type(pixel_values)}")
            
        image_features = self.clip.get_image_features(pixel_values)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        logits = self.classifier(image_features)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            
        return loss, logits

def collate_fn(batch):
    """Custom collate function to properly process the batch"""
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Convert images from list format to numpy arrays
    images = []
    labels = []
    
    for item in batch:
        # Convert image to numpy array if it's not already
        img = np.array(item['img']) if not isinstance(item['img'], np.ndarray) else item['img']
        
        # Ensure image is in correct format (H, W, C)
        if img.shape[0] == 3:  # If image is in (C, H, W) format
            img = np.transpose(img, (1, 2, 0))
            
        images.append(img)
        labels.append(item['label'])
    
    # Process images with CLIP processor
    inputs = processor(images=images, return_tensors="pt", padding=True)
    
    return {
        'pixel_values': inputs.pixel_values,
        'labels': torch.tensor(labels)
    }

# Setup data loaders with proper collate function
train_dataloader = DataLoader(
    preprocessed_train_ds,
    batch_size=8,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

val_dataloader = DataLoader(
    preprocessed_val_ds,
    batch_size=8,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

def train_model(model, train_dataloader, val_dataloader, num_epochs=6, learning_rate=5e-5, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=learning_rate)
    
    save_dir = os.path.abspath("./model_checkpoints")
    os.makedirs(save_dir, exist_ok=True)
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        total_loss = 0
        num_batches = 0
        
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            try:
                pixel_values = batch['pixel_values'].to(device)
                labels = batch['labels'].to(device)
                
                optimizer.zero_grad()
                loss, _ = model(pixel_values, labels)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                num_batches += 1
            except Exception as e:
                print(f"Error processing batch: {str(e)}")
                continue
        
        avg_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1} - Average training loss: {avg_loss:.4f}")
        
        # Validation loop
        model.eval()
        val_loss = 0
        val_batches = 0
        
        with torch.no_grad():
            for batch in val_dataloader:
                try:
                    pixel_values = batch['pixel_values'].to(device)
                    labels = batch['labels'].to(device)
                    loss, _ = model(pixel_values, labels)
                    val_loss += loss.item()
                    val_batches += 1
                except Exception as e:
                    print(f"Error processing validation batch: {str(e)}")
                    continue
        
        avg_val_loss = val_loss / val_batches
        print(f"Epoch {epoch + 1}: Val Loss = {avg_val_loss:.4f}")
        
        # Save checkpoint if it's the best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_path = os.path.join(save_dir, "classifier_weights.pt")
            torch.save(model.classifier.state_dict(), save_path)
            print(f"Saved best model to {save_path}")


# Could you share a sample of what your preprocessed_train_ds looks like?
# It should be something like this:
# class ImageDataset(Dataset):
#     def __init__(self, images, labels):
#         self.images = images
#         self.labels = labels
#     
#     def __len__(self):
#         return len(self.labels)
#     
#     def __getitem__(self, idx):
#         return {
#             'image': self.images[idx],
#             'label': self.labels[idx]
#         }

# Train model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CLIPForImageClassification(num_labels=10)
train_model(model, train_dataloader, val_dataloader, device=device)

# Load function remains the same
def load_model(num_labels=10):
    model = CLIPForImageClassification(num_labels=num_labels)
    save_path = os.path.join(os.path.abspath("./model_checkpoints"), "classifier_weights.pt")
    try:
        model.classifier.load_state_dict(torch.load(save_path))
        print("Model loaded successfully")
    except Exception as e:
        print(f"Error loading model: {str(e)}")
    return model

Using device: cpu












































































































































































































Epoch 1/6: 100%|██████████| 201/201 [06:00<00:00,  1.79s/it]


Epoch 1 - Average training loss: 2.2608
Epoch 1: Val Loss = 2.2308
Saved best model to c:\Users\ASUS\Desktop\shoppin-assignment\model_checkpoints\classifier_weights.pt


: 

In [ ]:
from transformers import CLIPFeatureExtractor

# Load the feature extractor for CLIP
feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")

c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
def preprocess_images(examples):
    # get batch of images
    images = examples['img']
    # convert to list of NumPy arrays of shape (C, H, W)
    images = [np.array(image, dtype=np.uint8) for image in images]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    # preprocess and add pixel_values
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']

    return examples

In [ ]:
top_labels = pd.DataFrame(filtered_df.groupby('articleType').size().reset_index().sort_values(0,ascending = False)[:11]['articleType'])
top_labels_list = sorted(list(top_labels['articleType']))
top_labels['label_num'] = top_labels['articleType'].apply(lambda x: top_labels_list.index(x))
top_labels

,articleType,label_num
7,Tshirts,7
4,Shirts,4
0,Casual Shoes,0
9,Watches,9
5,Sports Shoes,5
3,Kurtas,3
6,Tops,6
1,Handbags,1
2,Heels,2
8,Wallets,8


In [ ]:
from datasets import Features, ClassLabel, Array3D

# we need to define the features ourselves as both the img and pixel_values have a 3D shape 
features = Features({
    'label': ClassLabel(names = top_labels_list),
    'img': Array3D(dtype="int64", shape=(3,32,32)),
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
})

preprocessed_train_ds = train_ds.map(preprocess_images, batched=True, features=features)
preprocessed_val_ds = val_ds.map(preprocess_images, batched=True, features=features)
preprocessed_test_ds = test_ds.map(preprocess_images, batched=True, features=features)

Map:   0%|          | 0/1651 [00:00<?, ? examples/s]

Map:   0%|          | 0/515 [00:00<?, ? examples/s]

Map:   0%|          | 0/510 [00:00<?, ? examples/s]

In [ ]:
from transformers import CLIPModel, CLIPProcessor
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

class CLIPForImageClassification(nn.Module):
    def __init__(self, num_labels=10):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.num_labels = num_labels
        self.classifier = nn.Linear(512, num_labels)
        
        # Freeze CLIP
        for param in self.clip.parameters():
            param.requires_grad = False

    def forward(self, pixel_values, labels=None):
        # Ensure pixel_values is the right shape
        if not isinstance(pixel_values, torch.Tensor):
            raise ValueError(f"pixel_values must be a tensor, got {type(pixel_values)}")
            
        image_features = self.clip.get_image_features(pixel_values)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        logits = self.classifier(image_features)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            
        return loss, logits

def collate_fn(batch):
    """Custom collate function to properly process the batch"""
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Convert images from list format to numpy arrays
    images = []
    labels = []
    
    for item in batch:
        # Convert image to numpy array if it's not already
        img = np.array(item['img']) if not isinstance(item['img'], np.ndarray) else item['img']
        
        # Ensure image is in correct format (H, W, C)
        if img.shape[0] == 3:  # If image is in (C, H, W) format
            img = np.transpose(img, (1, 2, 0))
            
        images.append(img)
        labels.append(item['label'])
    
    # Process images with CLIP processor
    inputs = processor(images=images, return_tensors="pt", padding=True)
    
    return {
        'pixel_values': inputs.pixel_values,
        'labels': torch.tensor(labels)
    }

# Setup data loaders with proper collate function
train_dataloader = DataLoader(
    preprocessed_train_ds,
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

val_dataloader = DataLoader(
    preprocessed_val_ds,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

In [ ]:
def compute_accuracy(logits, labels):
    predictions = torch.argmax(logits, dim=1)
    return (predictions == labels).float().mean().item()

def train_model(model, train_dataloader, val_dataloader, num_epochs=6, learning_rate=5e-5, device="cuda", 
                start_epoch=0, save_dir="./model_checkpoints"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=learning_rate)
    
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Load the best metrics if resuming
    best_val_loss = float('inf')
    best_val_acc = 0.0
    if start_epoch > 0:
        try:
            checkpoint_path = os.path.join(save_dir, "best_classifier_weights.pt")
            model.classifier.load_state_dict(torch.load(checkpoint_path))
            print(f"Resumed from checkpoint: {checkpoint_path}")
        except Exception as e:
            print(f"Error loading checkpoint: {str(e)}")
            return
    
    for epoch in range(start_epoch, num_epochs):
        # Training loop
        model.train()
        total_loss = 0
        total_acc = 0
        num_batches = 0
        
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            try:
                pixel_values = batch['pixel_values'].to(device)
                labels = batch['labels'].to(device)
                
                optimizer.zero_grad()
                loss, logits = model(pixel_values, labels)
                loss.backward()
                optimizer.step()
                
                # Compute accuracy
                batch_acc = compute_accuracy(logits, labels)
                
                total_loss += loss.item()
                total_acc += batch_acc
                num_batches += 1
                
                # Save temporary checkpoint every 100 batches
                if num_batches % 100 == 0:
                    temp_path = os.path.join(save_dir, f"temp_checkpoint_epoch{epoch}_batch{num_batches}.pt")
                    torch.save(model.classifier.state_dict(), temp_path)
                    print(f"\nBatch {num_batches}: Loss = {loss.item():.4f}, Accuracy = {batch_acc:.4f}")
                    
            except Exception as e:
                print(f"Error processing batch: {str(e)}")
                continue
        
        avg_loss = total_loss / num_batches
        avg_acc = total_acc / num_batches
        print(f"\nEpoch {epoch + 1} Training:")
        print(f"Average Loss: {avg_loss:.4f}")
        print(f"Average Accuracy: {avg_acc:.4f}")
        
        # Validation loop
        model.eval()
        val_loss = 0
        val_acc = 0
        val_batches = 0
        
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                try:
                    pixel_values = batch['pixel_values'].to(device)
                    labels = batch['labels'].to(device)
                    loss, logits = model(pixel_values, labels)
                    
                    # Compute accuracy
                    batch_acc = compute_accuracy(logits, labels)
                    
                    val_loss += loss.item()
                    val_acc += batch_acc
                    val_batches += 1
                except Exception as e:
                    print(f"Error processing validation batch: {str(e)}")
                    continue
        
        avg_val_loss = val_loss / val_batches
        avg_val_acc = val_acc / val_batches
        print(f"\nEpoch {epoch + 1} Validation:")
        print(f"Average Loss: {avg_val_loss:.4f}")
        print(f"Average Accuracy: {avg_val_acc:.4f}")
        
        # Save checkpoint if it's the best model (using both loss and accuracy)
        if avg_val_acc > best_val_acc:
            best_val_acc = avg_val_acc
            save_path = os.path.join(save_dir, "best_classifier_weights.pt")
            torch.save(model.classifier.state_dict(), save_path)
            print(f"Saved best model (accuracy) to {save_path}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_path = os.path.join(save_dir, "best_classifier_weights_loss.pt")
            torch.save(model.classifier.state_dict(), save_path)
            print(f"Saved best model (loss) to {save_path}")
        
        # Save epoch checkpoint
        epoch_path = os.path.join(save_dir, f"checkpoint_epoch{epoch+1}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.classifier.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'accuracy': avg_acc,
            'val_loss': avg_val_loss,
            'val_accuracy': avg_val_acc,
            'best_val_loss': best_val_loss,
            'best_val_accuracy': best_val_acc
        }, epoch_path)
        
        print('-' * 50)

    return {
        'best_val_loss': best_val_loss,
        'best_val_accuracy': best_val_acc
    }

# To resume training:
model = CLIPForImageClassification(num_labels=10)
start_epoch = 3  # Change this to the epoch you want to resume from
save_dir = "./model_checkpoints"  # Your checkpoint directory

# Train model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

results = train_model(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    start_epoch=start_epoch,
    save_dir=save_dir
)

print("\nTraining completed!")
print(f"Best validation loss: {results['best_val_loss']:.4f}")
print(f"Best validation accuracy: {results['best_val_accuracy']:.4f}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22224\88305022.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.classifier.load_state_dict(torch.load(checkpoint_path))


Using device: cpu
Resumed from checkpoint: ./model_checkpoints\best_classifier_weights.pt


Epoch 4/6:  24%|██▍       | 100/413 [03:23<10:35,  2.03s/it]


Batch 100: Loss = 2.1044, Accuracy = 0.5000


Epoch 4/6:  48%|████▊     | 200/413 [06:45<07:11,  2.03s/it]

...


Batch 200: Loss = 1.7320, Accuracy = 0.7500


Epoch 6/6:  73%|███████▎  | 300/413 [11:56<06:03,  3.21s/it]


Batch 300: Loss = 1.7023, Accuracy = 0.7500


Epoch 6/6:  92%|█████████▏| 381/413 [16:14<01:45,  3.29s/it]

: 